## imports

In [1]:
import torch
from modules.DRN.DRNSeg import DRNSeg
import argparse , os ,sys
import numpy as np
import torchvision.transforms as transforms
from PIL import Image
from utils.tools import TimerBlock , SeedSetter
from dataset import Photoshopped_Faces
from losses import L_tot
from torch.autograd import Function, Variable


print("y")

y


## Args
TO DO: chagne it to parser instead

In [2]:
class Args():
    def __init__(self,model_path,gpu_id=0,lr=1e-4,b1=0.9,b2=0.999):
        self.lr = lr
        self.b1 = b1
        self.b2 = b2
        self.gpu_id=gpu_id
        self.model_path =model_path
        
args = Args("../checkpoints/DRN/local.pth")       

## Train

In [3]:
from utils.geometry import forward_backward_consistency_check
from utils.tools import grouped_files_from_dir_list,SeedSetter,toTensor
from utils.tools import read_gen , resize_gen
from torch.utils.data import Dataset
from torchvision.transforms import v2
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm


sources = "../sources"
images_dir = sources+"/images"
flow_res = sources+"/flows"
m_dir = flow_res + "/M"
flow_res_fwd = flow_res + "/fwd"
flow_res_bwd = flow_res + "/bwd"
mod = images_dir + "/modified"
ref = images_dir + "/reference"
dir_lst = [ref,mod]
normalize = v2.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
ds = Photoshopped_Faces(dir_lst,size = (128,128),transform=normalize)
train ,val ,test= torch.utils.data.random_split(ds, [8, 92,899])
train_loader = DataLoader(
    dataset=train,
    batch_size=2,
    shuffle=True,
    drop_last=True
)
val_loader = DataLoader(
    dataset=val,
    batch_size=2,
    shuffle=True,
    drop_last=True
)

epoch = 1 
num_epochs = 20




In [4]:
#check if cuda is available:
if torch.cuda.is_available():
    device = 'cuda:{}'.format(args.gpu_id)
else:
    device = 'cpu'
    
#pretrained DRN model:
model = DRNSeg(2)
state_dict = torch.load(args.model_path, map_location=device)
model.load_state_dict(state_dict['model'])

#set optimizer
optimizer = torch.optim.Adam(model.parameters(),args.lr,(args.b1,args.b2))
loss_total = L_tot(s_epe = 1.0 , s_ms = 15.0 , s_rec = 1.5 , flow_factor=1.05)

best_val_loss = float('inf')
best_model_path = 'best_model.pth'

num_epochs = 5




for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    train_loss = 0.0

    # Training loop with tqdm progress bar
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training", leave=False):
        model.cuda()
        batch = [t.cuda() for t in batch]
        batch = [Variable(t.data, requires_grad=True) for t in batch]
        Xo , Xm , Umo , M = batch
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(Xm).to(device=device)  # Forward pass
        outputs = Variable(outputs.data, requires_grad=True)
        print("computer loss:")
        loss = loss_total(outputs,Xo,Xm,Umo,M) # Compute loss
        print("backward:")
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        train_loss += loss.item()  # Accumulate loss

    # Validation phase
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    with torch.no_grad():  # Disable gradient computation
        for batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation", leave=False):
            model.cuda()
            batch = [t.cuda() for t in batch]
            Xo , Xm , Umo , M = batch
            outputs = model(Xm).to(device=device)  # Forward pass
            loss = loss_total(outputs,Xo,Xm,Umo,M)  # Compute loss
            val_loss += loss.item()  # Accumulate loss

    val_loss /= len(val_loader)  # Calculate average validation loss

    # Save the best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), best_model_path)  # Save best model checkpoint
        print(f'Saved best model with loss: {best_val_loss:.4f}')

    # Print epoch summary
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss/len(train_loader):.4f}, Validation Loss: {val_loss:.4f}')


Epoch 1/5 - Training:   0%|                                                                       | 0/4 [00:00<?, ?it/s]/home/din/miniconda3/envs/buna/lib/python3.10/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1712608935911/work/aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


computer loss:


backward:


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn